In [1]:
# Datenzugriff
from configs.path_manager import get_path

# Utils
from src.utils.rolling_process.rolling_filter import *
import src.utils.visualization.gantt_diagram as gantt
from src.utils.analysis_basics import check_constrains as check
from src.utils.analysis_basics.check_scheduled_jobs import get_jobs_with_lateness_metrics

# Simulation
from src.simulation.ProductionRollingSimulation import ProductionSimulation

# Extern
import pandas as pd

pd.set_option('display.min_rows', 12)
pd.set_option('display.max_rows', 16)

In [2]:
# Harte Abbruchbedingung für die Scheduler
max_time = 60*30     # 30 min

# Simulation
simulation = ProductionSimulation(sigma = 0.15)

In [3]:
import pandas as pd
from collections import defaultdict
import random

def giffler_thompson_schedule(
        df_jssp: pd.DataFrame,
        df_arrivals_deadlines: pd.DataFrame,
        priority_rule: str = "SPT",
        log_on: bool = True
) -> pd.DataFrame:
    """
    Giffler-Thompson-Algorithmus mit Ready Time (für Planung) und Arrival (für Ausgabe).
    Rückgabe mit Lateness, Tardiness, Earliness.

    Rückgabe:
    - df_schedule: DataFrame mit Zeitplan und Auswertung
    - makespan: maximale Endzeit
    - total_abs_lateness: Summe der absoluten Lateness der letzten Operationen je Job
    """

    if log_on:
        print(f'Giffler-Thompson-Algorithmus mit Prioritätsregel \"{priority_rule}\" gestartet ...\n')

    df = df_jssp.copy().sort_values(['Job', 'Operation']).reset_index(drop=True)
    machines = set(df['Machine'])
    jobs = df['Job'].unique()

    ready_time = df_arrivals_deadlines.set_index("Job")["Ready Time"].to_dict()
    arrival = df_arrivals_deadlines.set_index("Job")["Arrival"].to_dict()
    deadline = df_arrivals_deadlines.set_index("Job")["Deadline"].to_dict()

    job_op_ready = {job: 0 for job in jobs}
    machine_available = {m: 0 for m in machines}
    job_op_index = {job: 0 for job in jobs}

    job_ops = defaultdict(list)
    for _, row in df.iterrows():
        job_ops[row['Job']].append((row['Operation'], row['Machine'], row['Processing Time']))

    result = []
    remaining = sum(len(v) for v in job_ops.values())

    while remaining > 0:
        candidates = []
        for job in jobs:
            idx = job_op_index[job]
            if idx < len(job_ops[job]):
                op_id, m, d = job_ops[job][idx]
                est = max(job_op_ready[job], machine_available[m], ready_time[job])
                candidates.append((est, d, job, op_id, m))

        min_est = min(c[0] for c in candidates)
        conflict_ops = [c for c in candidates if c[0] == min_est]

        if priority_rule == "SPT":
            selected = min(conflict_ops, key=lambda x: x[1])
        elif priority_rule == "EDF" or priority_rule == "EDD":
            selected = min(conflict_ops, key=lambda x: deadline[x[2]])  # x[2] ist der Job
        else:  # RANDOM oder unbekannt
            selected = random.choice(conflict_ops)

        est, d, job, op_id, m = selected
        start = est
        end = start + d

        result.append({
            'Job': job,
            'Operation': op_id,
            'Machine': m,
            'Start': start,
            'Processing Time': d,
            'End': end
        })

        job_op_ready[job] = end
        machine_available[m] = end
        job_op_index[job] += 1
        remaining -= 1

    # Berechnung von Lateness, Tardiness, Earliness
    for row in result:
        job = row['Job']
        lateness_val = row['End'] - deadline[job]
        row.update({
            "Arrival": arrival[job],
            "Deadline": deadline[job],
            "Lateness": lateness_val,
            "Tardiness": max(0, lateness_val),
            "Earliness": max(0, -lateness_val)
        })

    df_schedule = pd.DataFrame(result).sort_values(['Start', 'Job', 'Operation']).reset_index(drop=True)

    last_ops = df_schedule.groupby("Job")["Operation"].max()
    last_op_index_tuples = list(zip(last_ops.index, last_ops.values))
    mask_last_ops = df_schedule.set_index(["Job", "Operation"]).index.isin(last_op_index_tuples)
    total_abs_lateness = df_schedule[mask_last_ops]["Lateness"].abs().sum()

    makespan = df_schedule['End'].max()

    if log_on:
        print("\nPlan-Informationen:")
        print(f"  Anzahl Jobs            : {len(jobs)}")
        print(f"  Anzahl Maschinen       : {len(machines)}")
        print(f"  Anzahl Operationen     : {len(df_schedule)}")
        print(f"  Makespan               : {makespan}")
        print(f"  Summe Absolute Lateness: {total_abs_lateness}")

    return df_schedule, makespan, total_abs_lateness

In [4]:
def giffler_thompson_schedule2(
        df_jssp: pd.DataFrame,
        df_arrivals_deadlines: pd.DataFrame,
        priority_rule: str = "SPT",
        df_execution: pd.DataFrame = None,
        reschedule_start: float = 1440.0,
        log_on: bool = True
):
    if log_on:
        print(f"Giffler-Thompson mit Prioritätsregel '{priority_rule}' gestartet.\n")

    df = df_jssp.copy().sort_values(['Job', 'Operation']).reset_index(drop=True)
    machines = set(df['Machine'])
    jobs = df['Job'].unique()

    ready_time = df_arrivals_deadlines.set_index("Job")["Ready Time"].to_dict()
    arrival = df_arrivals_deadlines.set_index("Job")["Arrival"].to_dict()
    deadline = df_arrivals_deadlines.set_index("Job")["Deadline"].to_dict()

    executed_ends = {}
    executed_last_op = {}
    fixed_machine_blocks = defaultdict(list)
    if df_execution is not None and not df_execution.empty:
        executed_ends = df_execution.groupby("Job")["End"].max().to_dict()
        executed_last_op = df_execution.groupby("Job")["Operation"].max().to_dict()
        for _, row in df_execution.iterrows():
            fixed_machine_blocks[row["Machine"]].append((row["Start"], row["End"]))

    job_op_index = {
        job: executed_last_op[job] + 1 if job in executed_last_op else 0
        for job in jobs
    }

    job_op_ready = {
        job: executed_ends[job] if job in executed_ends else 0
        for job in jobs
    }

    machine_available = {m: 0 for m in machines}

    job_ops = defaultdict(list)
    for _, row in df.iterrows():
        job_ops[row['Job']].append((row['Operation'], row['Machine'], row['Processing Time']))

    result = []
    while True:
        candidates = []
        for job in jobs:
            idx = job_op_index[job]
            if idx < len(job_ops[job]):
                op_id, m, d = job_ops[job][idx]

                base_time = max(reschedule_start, job_op_ready[job])
                est = max(machine_available[m], base_time)

                # Blockierende Maschinenzeiten berücksichtigen
                if m in fixed_machine_blocks:
                    while any(s <= est < e for s, e in fixed_machine_blocks[m]):
                        est = max(e for s, e in fixed_machine_blocks[m] if s <= est < e)

                # ✅ TECHNOLOGIE-PRÜFUNG: Ist Vorgänger eingeplant?
                if op_id > 0:
                    prev_end = None
                    for r in result:
                        if r["Job"] == job and r["Operation"] == op_id - 1:
                            prev_end = r["End"]
                            break
                    if prev_end is None and df_execution is not None and not df_execution.empty:
                        prev_df = df_execution[
                            (df_execution["Job"] == job) & (df_execution["Operation"] == op_id - 1)
                            ]
                        if not prev_df.empty:
                            prev_end = prev_df.iloc[0]["End"]

                    if prev_end is None:
                        continue  # Vorgänger nicht bekannt → überspringen

                    est = max(est, prev_end)

                candidates.append((est, d, job, op_id, m))

        if not candidates:
            if log_on:
                print("⚠️ Keine einplanbaren Kandidaten mehr – evtl. Vorgänger- oder Datenfehler.\n")
                for job in jobs:
                    idx = job_op_index[job]
                    if idx < len(job_ops[job]):
                        print(f"  Job {job} wartet auf Operation {idx}")
            break

        min_est = min(c[0] for c in candidates)
        conflict_ops = [c for c in candidates if c[0] == min_est]

        if priority_rule == "SPT":
            selected = min(conflict_ops, key=lambda x: x[1])
        elif priority_rule in ["EDF", "EDD"]:
            selected = min(conflict_ops, key=lambda x: deadline[x[2]])
        else:
            selected = random.choice(conflict_ops)

        est, d, job, op_id, m = selected
        start = est
        end = start + d

        result.append({
            'Job': job,
            'Operation': op_id,
            'Machine': m,
            'Start': start,
            'Processing Time': d,
            'End': end
        })

        job_op_ready[job] = end
        machine_available[m] = end
        job_op_index[job] += 1

    for row in result:
        job = row['Job']
        lateness_val = row['End'] - deadline[job]
        row.update({
            "Arrival": arrival[job],
            "Deadline": deadline[job],
            "Lateness": lateness_val,
            "Tardiness": max(0, lateness_val),
            "Earliness": max(0, -lateness_val)
        })

    df_schedule = pd.DataFrame(result)

    last_ops = df_schedule.groupby("Job")["Operation"].max()
    last_op_index_tuples = list(zip(last_ops.index, last_ops.values))
    mask_last_ops = df_schedule.set_index(["Job", "Operation"]).index.isin(last_op_index_tuples)
    total_abs_lateness = df_schedule[mask_last_ops]["Lateness"].abs().sum()
    makespan = df_schedule["End"].max()

    if df_execution is not None and not df_execution.empty:
        df_execution = df_execution.rename(columns={"Simulated Processing Time": "Processing Time"})
        df_mix = pd.concat([df_execution, df_schedule], ignore_index=True)
    else:
        df_mix = df_schedule.copy()

    # Diagnose: fehlende Operationen
    expected_ops = df.groupby("Job")["Operation"].count().to_dict()
    scheduled_ops = df_schedule.groupby("Job")["Operation"].count().to_dict()
    missing_jobs = {
        job: expected_ops[job] - scheduled_ops.get(job, 0)
        for job in expected_ops
        if expected_ops[job] != scheduled_ops.get(job, 0)
    }

    if log_on:
        print("\nPlan-Informationen:")
        print(f"  Anzahl Jobs            : {len(jobs)}")
        print(f"  Anzahl Maschinen       : {len(machines)}")
        print(f"  Anzahl Operationen     : {len(df_schedule)}")
        print(f"  Makespan               : {makespan}")
        print(f"  Summe Absolute Lateness: {total_abs_lateness}")
        if missing_jobs:
            print(f"\n⚠️ Fehlende Operationen:")
            for job, count in missing_jobs.items():
                print(f"  {job}: {count} nicht eingeplant")

    return df_schedule, makespan, total_abs_lateness, df_mix


def giffler_thompson_schedule2(
        df_jssp: pd.DataFrame,
        df_arrivals_deadlines: pd.DataFrame,
        priority_rule: str = "SPT",
        df_execution: pd.DataFrame = None,
        reschedule_start: float = 1440.0,
        log_on: bool = True
):
    if log_on:
        print(f"Giffler-Thompson mit Prioritätsregel '{priority_rule}' gestartet.\n")

    df = df_jssp.copy().sort_values(['Job', 'Operation']).reset_index(drop=True)
    machines = set(df['Machine'])
    jobs = df['Job'].unique()

    ready_time = df_arrivals_deadlines.set_index("Job")["Ready Time"].to_dict()
    arrival = df_arrivals_deadlines.set_index("Job")["Arrival"].to_dict()
    deadline = df_arrivals_deadlines.set_index("Job")["Deadline"].to_dict()

    executed_ends = {}
    executed_last_op = {}
    fixed_machine_blocks = defaultdict(list)
    if df_execution is not None and not df_execution.empty:
        executed_ends = df_execution.groupby("Job")["End"].max().to_dict()
        executed_last_op = df_execution.groupby("Job")["Operation"].max().to_dict()
        for _, row in df_execution.iterrows():
            fixed_machine_blocks[row["Machine"]].append((row["Start"], row["End"]))

    job_op_index = {
        job: executed_last_op[job] + 1 if job in executed_last_op else 0
        for job in jobs
    }

    job_op_ready = {
        job: executed_ends[job] if job in executed_ends else 0
        for job in jobs
    }

    machine_available = {m: 0 for m in machines}

    job_ops = defaultdict(list)
    for _, row in df.iterrows():
        job_ops[row['Job']].append((row['Operation'], row['Machine'], row['Processing Time']))

    result = []
    while True:
        candidates = []
        for job in jobs:
            idx = job_op_index[job]
            if idx < len(job_ops[job]):
                op_id, m, d = job_ops[job][idx]

                base_time = max(reschedule_start, job_op_ready[job])
                est = max(machine_available[m], base_time)

                if m in fixed_machine_blocks:
                    while any(s <= est < e for s, e in fixed_machine_blocks[m]):
                        est = max(e for s, e in fixed_machine_blocks[m] if s <= est < e)

                # ✅ Nur wenn df_execution vorhanden ist: Technologieprüfung
                if op_id > 0 and df_execution is not None and not df_execution.empty:
                    prev_end = None
                    for r in result:
                        if r["Job"] == job and r["Operation"] == op_id - 1:
                            prev_end = r["End"]
                            break
                    if prev_end is None:
                        prev_df = df_execution[
                            (df_execution["Job"] == job) & (df_execution["Operation"] == op_id - 1)
                            ]
                        if not prev_df.empty:
                            prev_end = prev_df.iloc[0]["End"]

                    if prev_end is None:
                        continue  # blockieren, aber nur bei aktivem df_execution

                    est = max(est, prev_end)

                candidates.append((est, d, job, op_id, m))

        if not candidates:
            if log_on:
                print("⚠️ Keine einplanbaren Kandidaten mehr – evtl. Vorgänger- oder Datenfehler.\n")
                for job in jobs:
                    idx = job_op_index[job]
                    if idx < len(job_ops[job]):
                        print(f"  Job {job} wartet auf Operation {idx}")
            break

        min_est = min(c[0] for c in candidates)
        conflict_ops = [c for c in candidates if c[0] == min_est]

        if priority_rule == "SPT":
            selected = min(conflict_ops, key=lambda x: x[1])
        elif priority_rule in ["EDF", "EDD"]:
            selected = min(conflict_ops, key=lambda x: deadline[x[2]])
        else:
            selected = random.choice(conflict_ops)

        est, d, job, op_id, m = selected
        start = est
        end = start + d

        result.append({
            'Job': job,
            'Operation': op_id,
            'Machine': m,
            'Start': start,
            'Processing Time': d,
            'End': end
        })

        job_op_ready[job] = end
        machine_available[m] = end
        job_op_index[job] += 1

    for row in result:
        job = row['Job']
        lateness_val = row['End'] - deadline[job]
        row.update({
            "Arrival": arrival[job],
            "Deadline": deadline[job],
            "Lateness": lateness_val,
            "Tardiness": max(0, lateness_val),
            "Earliness": max(0, -lateness_val)
        })

    df_schedule = pd.DataFrame(result)

    last_ops = df_schedule.groupby("Job")["Operation"].max()
    last_op_index_tuples = list(zip(last_ops.index, last_ops.values))
    mask_last_ops = df_schedule.set_index(["Job", "Operation"]).index.isin(last_op_index_tuples)
    total_abs_lateness = df_schedule[mask_last_ops]["Lateness"].abs().sum()
    makespan = df_schedule["End"].max()

    if df_execution is not None and not df_execution.empty:
        df_execution = df_execution.rename(columns={"Simulated Processing Time": "Processing Time"})
        df_mix = pd.concat([df_execution, df_schedule], ignore_index=True)
    else:
        df_mix = df_schedule.copy()

    # Diagnose: fehlende Operationen
    expected_ops = df.groupby("Job")["Operation"].count().to_dict()
    scheduled_ops = df_schedule.groupby("Job")["Operation"].count().to_dict()
    missing_jobs = {
        job: expected_ops[job] - scheduled_ops.get(job, 0)
        for job in expected_ops
        if expected_ops[job] != scheduled_ops.get(job, 0)
    }

    if log_on:
        print("\nPlan-Informationen:")
        print(f"  Anzahl Jobs            : {len(jobs)}")
        print(f"  Anzahl Maschinen       : {len(machines)}")
        print(f"  Anzahl Operationen     : {len(df_schedule)}")
        print(f"  Makespan               : {makespan}")
        print(f"  Summe Absolute Lateness: {total_abs_lateness}")
        if missing_jobs:
            print(f"\n⚠️ Fehlende Operationen:")
            for job, count in missing_jobs.items():
                print(f"  {job}: {count} nicht eingeplant")

    return df_schedule, makespan, total_abs_lateness, df_mix

## I. Laden des fertigen Job-Shop Scheduling Problems und der dazugehörigen Produktionsauftragsinformationen

In [5]:
basic_data_path = get_path("data", "basic")
df_jssp = pd.read_csv(basic_data_path / "jssp_final.csv")
df_jssp

,Job,Routing_ID,Operation,Machine,Processing Time
0,J25-0000,8,0,M00,76
1,J25-0000,8,1,M01,69
2,J25-0000,8,2,M03,76
3,J25-0000,8,3,M05,51
4,J25-0000,8,4,M02,85
5,J25-0000,8,5,M09,11
...,...,...,...,...,...
72494,J25-7249,4,4,M03,26
72495,J25-7249,4,5,M04,69
72496,J25-7249,4,6,M08,21


In [6]:
df_jobs_times = pd.read_csv(basic_data_path / "jobs_times_final.csv")
df_jobs_times

,Job,Routing_ID,Arrival,Ready Time,End,Job Processing Time,Deadline
0,J25-0000,8,79,1440,2704.0,597,3660
1,J25-0001,6,129,1440,2336.0,416,2338
2,J25-0002,0,198,1440,2925.0,395,3744
3,J25-0003,2,243,1440,2801.0,568,3379
4,J25-0004,3,287,1440,2697.0,655,4856
5,J25-0005,1,498,1440,2630.0,510,3106
...,...,...,...,...,...,...,...
7244,J25-7244,3,518085,518400,520251.0,655,520389
7245,J25-7245,9,518089,518400,520296.0,540,519640
7246,J25-7246,6,518116,518400,520027.0,416,520157


## Tag 1
- Am Tag 0 kommen die ersten Produktionsaufträge an, somit kann noch nichts geplant werden
- Am Tag 1 findet die initiale Planung statt

In [7]:
day_length = 1440

day_numb = 1
day_start = day_length*day_numb
day_end = day_start + day_length
print(f"Tag {day_numb:02d}: [{day_start}, {day_end})")

Tag 01: [1440, 2880)


In [8]:
# "neue" und unerledigte Jobs
df_jobs_times_current = get_current_jobs(
    df_jobs_times, df_previous_not_started = None, ready_time=day_start
)
df_jobs_times_current

,Job,Routing_ID,Arrival,Ready Time,End,Job Processing Time,Deadline
0,J25-0000,8,79,1440,2704.0,597,3660
1,J25-0001,6,129,1440,2336.0,416,2338
2,J25-0002,0,198,1440,2925.0,395,3744
3,J25-0003,2,243,1440,2801.0,568,3379
4,J25-0004,3,287,1440,2697.0,655,4856
5,J25-0005,1,498,1440,2630.0,510,3106
6,J25-0006,9,790,1440,2905.0,540,3539
7,J25-0007,5,925,1440,2930.0,496,2929
8,J25-0008,4,988,1440,2986.0,393,3541
9,J25-0009,7,1004,1440,3045.0,539,3578


In [9]:
df_jssp_current = filter_current_jssp(df_jssp, df_jobs_times_current)
df_jssp_current

,Job,Routing_ID,Operation,Machine,Processing Time
0,J25-0000,8,0,M00,76
1,J25-0000,8,1,M01,69
2,J25-0000,8,2,M03,76
3,J25-0000,8,3,M05,51
4,J25-0000,8,4,M02,85
5,J25-0000,8,5,M09,11
...,...,...,...,...,...
154,J25-0015,5,4,M08,48
155,J25-0015,5,5,M09,72
156,J25-0015,5,6,M00,47


### Scheduling

#### a) Summe der absoluten Lateness minimieren

$$
\min \sum_{j \in J} \left| C_j - d_j \right|
$$

- $J$: Menge aller Jobs  
- $C_j$: tatsächliche Endzeit (Completion Time) von Job $j$  
- $d_j$: Deadline von Job $j$  
- $\left| C_j - d_j \right|$: absolute Abweichung zur Deadline

**Erklärung:**  
Diese Zielfunktion minimiert die **Summe aller Abweichungen zur Deadline**, unabhängig davon, ob ein Job zu früh oder zu spät fertig wird.  
Sie fördert eine **gleichmäßige, zentrierte Fertigstellung** aller Jobs um ihre Soll-Zeitpunkte (Deadlines).

In [10]:
# Scheduling
df_schedule, makespan, total_abs_lateness = giffler_thompson_schedule(
    df_jssp=df_jssp_current,
    df_arrivals_deadlines=df_jobs_times_current,
    priority_rule="EDF",
    log_on=True
)

df_schedule

Giffler-Thompson-Algorithmus mit Prioritätsregel "EDF" gestartet ...


Plan-Informationen:
  Anzahl Jobs            : 16
  Anzahl Maschinen       : 10
  Anzahl Operationen     : 160
  Makespan               : 3215
  Summe Absolute Lateness: 13135


,Job,Operation,Machine,Start,Processing Time,End,Arrival,Deadline,Lateness,Tardiness,Earliness
0,J25-0001,0,M01,1440,46,1486,129,2338,-852,0,852
1,J25-0005,0,M00,1440,43,1483,498,3106,-1623,0,1623
2,J25-0007,0,M02,1440,84,1524,925,2929,-1405,0,1405
3,J25-0011,0,M00,1483,43,1526,1030,3210,-1684,0,1684
4,J25-0013,0,M01,1486,81,1567,1192,2567,-1000,0,1000
5,J25-0015,0,M02,1524,84,1608,1436,2981,-1373,0,1373
...,...,...,...,...,...,...,...,...,...,...,...
154,J25-0004,5,M08,2915,52,2967,287,4856,-1889,0,1889
155,J25-0012,9,M04,2953,33,2986,1103,4385,-1399,0,1399
156,J25-0004,6,M07,2967,85,3052,287,4856,-1804,0,1804


In [11]:
df_last_ops = get_jobs_with_lateness_metrics(df_schedule)
df_last_ops

,Job,Operation,Machine,Start,Processing Time,End,Arrival,Deadline,Lateness,Tardiness,Earliness
144,J25-0000,9,M08,2746,74,2820,79,3660,-840,0,840
67,J25-0001,9,M04,2061,55,2116,129,2338,-222,0,222
151,J25-0002,9,M09,2887,21,2908,198,3744,-836,0,836
116,J25-0003,9,M04,2406,33,2439,243,3379,-940,0,940
159,J25-0004,9,M05,3172,43,3215,287,4856,-1641,0,1641
108,J25-0005,9,M08,2320,30,2350,498,3106,-756,0,756
128,J25-0006,9,M07,2537,45,2582,790,3539,-957,0,957
57,J25-0007,9,M07,1990,25,2015,925,2929,-914,0,914
146,J25-0008,9,M06,2764,53,2817,988,3541,-724,0,724
148,J25-0009,9,M03,2787,79,2866,1004,3578,-712,0,712


In [12]:
# show.count_column_grouped(last_ops, "Tardiness", max_val = 840, steps= 120)

SyntaxError: invalid syntax (3457752569.py, line 1)

In [ ]:
gantt.get_plot(df_schedule, perspective="Machine")
check.is_machine_conflict_free(df_schedule)
check.is_operation_sequence_correct(df_schedule)
check.is_job_timing_correct(df_schedule)
check.is_start_correct(df_schedule)

### Simulation

In [ ]:
simulation.run(dframe_schedule_plan = df_schedule, start_time = day_start, end_time=day_end)

In [ ]:
df_execution = simulation.get_finished_operations_df()
df_execution

In [ ]:
gantt.get_plot(df_execution, perspective="Machine")

In [ ]:
df_active = simulation.get_active_operations_df()
df_active

In [ ]:
if df_active is not None:
    gantt.get_plot(df_active, perspective="Machine", duration_column="Planned Duration")

In [ ]:
df_not_started = simulation.get_not_started_operations_df(df_schedule)
df_not_started

## Tag 2
- hier findet die Neuplanung statt, also das Scheduling mit bestimmter Berücksichtigung der Simulation

In [ ]:
day_numb = 2
day_start = day_length*day_numb
day_end = day_start + day_length
print(f"Tag {day_numb:02d}: [{day_start}, {day_end})")

In [ ]:
# "neue" und unerledigte Jobs
df_jobs_times_current = get_current_jobs(df_jobs_times, df_not_started, ready_time=day_start)
df_jobs_times_current

In [ ]:
df_jssp_current = filter_current_jssp(
    df_jssp = df_jssp,
    df_jobs_times_current = df_jobs_times_current,
    exclusion_dataframes_list = [df_active, df_execution]
)
df_jssp_current

### Neuplanung

In [ ]:
df_reschedule, total_abs_lateness, df_mix = giffler_thompson_schedule2(
    df_jssp=df_jssp_current,
    df_arrivals_deadlines=df_jobs_times_current,
    priority_rule="EDF",
    df_execution=None,
    reschedule_start = 2880,
    log_on=True
)
df_reschedule

In [ ]:
df_last_ops = get_jobs_with_lateness_metrics(df_reschedule)
df_last_ops

In [ ]:
# show.count_column_grouped(last_ops, "Tardiness", max_val = 840, steps= 120)

In [ ]:
gantt.get_plot(df_reschedule, perspective="Machine")
check.is_machine_conflict_free(df_reschedule)
check.is_operation_sequence_correct(df_reschedule)
check.is_job_timing_correct(df_reschedule)
check.is_start_correct(df_reschedule)

#### Check der Bedingungen des kombinierten Datensatzes aus den Simulationsdaten mit dem neuen Schedule

In [ ]:

columns = ["Job", "Operation", "Machine", "Arrival", "Start", "Processing Time", "End"]
df_combined = pd.concat([df_execution, df_active[columns], df_reschedule[columns]], ignore_index=True)
gantt.get_plot(df_combined, perspective="Machine")
check.is_machine_conflict_free(df_combined)
check.is_operation_sequence_correct(df_combined)
check.is_job_timing_correct(df_combined)
check.is_start_correct(df_combined)


### Simulation

In [ ]:
simulation.run(dframe_schedule_plan = df_reschedule, start_time = day_start, end_time = day_end)

In [ ]:
df_execution2 = simulation.get_finished_operations_df()
df_execution2

In [ ]:
gantt.get_plot(df_execution2, perspective="Machine")

In [ ]:
df_active2 = simulation.get_active_operations_df()
df_active2

In [ ]:
df_not_started2 = simulation.get_not_started_operations_df(df_reschedule)
df_not_started2

#### Check der Bedingungen des kombinierten Datensatzes aus den beiden Simulationsdaten

In [ ]:
df_combined = pd.concat([df_execution, df_execution2], ignore_index=True)
gantt.get_plot(df_combined, perspective="Machine")
check.is_machine_conflict_free(df_combined)
check.is_operation_sequence_correct(df_combined)
check.is_job_timing_correct(df_combined)
check.is_start_correct(df_combined)